In [1]:
import numpy as np
import cv2
import math
import time

In [21]:
def find_mask(frame):
    mask = cv2.inRange(frame, (0,0,0), (254, 254, 254))
    output = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]

    dots = []

    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        t = stats[i, cv2.CC_STAT_TOP]
        l = stats[i, cv2.CC_STAT_LEFT]
        if (a > 10 and a < 400 and w/h < 3 and h/w < 3):
            dots.append((l + w//2, t + h//2))
            cv2.rectangle(frame, (l, t), (l + w, t + h), (0, 0, 255), 3)

    return mask, dots

In [3]:
def draw_line(frame, a, b, xmin, xmax, color = (200, 30, 30)):
    y1 = int(xmin * a + b)
    y2 = int(xmax * a + b)
    cv2.line(frame, (int(xmin), int(y1)), (int(xmax), int(y2)), color, 3)

In [4]:
def draw_line_r_theta(frame, r, theta, xmin, xmax, color = (200, 30, 30)):
    a = math.tan(theta)
    x0 = r * math.cos(theta + math.pi/2)
    y0 = r * math.sin(theta + math.pi/2)
    b = y0 - xo * a
    draw_line(frame, a, b, xmin, xmax, color)

In [5]:
def repack_data(dots):
    n = len(dots)
    x = np.zeros((n))
    y = np.zeros((n))
    for i in range(n):
        x[i], y[i] = dots[i]
    xmin = min(x)
    xmax = max(x)
    return n, x, y, xmin, xmax

In [6]:
def calc_distance(a, b, dot):
    x, y = dot
    xp = (y*a + x - b*a) / (a*a + 1)
    yp = a*xp + b
    return math.sqrt((xp - x)**2 + (yp - y)**2)

In [7]:
def r_theta_by_a_b(a, b):
    return b / math.sqrt(a*a + 1), math.atan(a) # r, theta

In [8]:
def LSE(dots):
    n, x, y, xmin, xmax = repack_data(dots)

    Y = sum(y)
    X = sum(x)
    XY = sum(x*y)
    XX = sum(x*x)
    
    b_hat = (Y*XX - XY*X)/(n*XX - X*X)
    a_hat = (XY - b_hat*X) / XX
    
    return a_hat, b_hat, xmin, xmax

In [22]:
img = cv2.imread(r"dots.png")

mask, dots = find_mask(img)
ah, bh, xmin, xmax = LSE(dots)
draw_line(img, ah, bh, xmin, xmax)

cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()